In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role() 

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
!pwd

In [ ]:
model_id = "google/flan-t5-xxl"
model_name = model_id.split("/")[-1]

In [ ]:
with open("model/model_name.txt", "w") as f:
    f.write(model_id)

In [ ]:
import os
filePath = 'model/model.tar.gz'

if os.path.exists(filePath):
    os.remove(filePath)

In [ ]:
!tar zcvf model/model.tar.gz model/*

In [ ]:
s3_location = f"s3://{sess.default_bucket()}/{model_name}/model.tar.gz"

In [ ]:
!aws s3 cp model/model.tar.gz $s3_location

In [ ]:
from sagemaker.utils import name_from_base
from sagemaker.huggingface.model import HuggingFaceModel

endpoint_name = name_from_base(model_name)

huggingface_model = HuggingFaceModel(
    model_data=s3_location,
    role=role,
    transformers_version="4.17",
    pytorch_version="1.10",
    py_version='py38',
)

In [ ]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    endpoint_name=endpoint_name,
)

In [ ]:
prompt = "This is a test to see if"

In [ ]:
data = {
    "inputs": prompt,
    "do_sample": False,
}

res = predictor.predict(data=data)
print(res)